<a href="https://colab.research.google.com/github/raj-vijay/da/blob/master/09_Constraint_Propagation_Propagator_Iteration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Constraint Propagation – Propagator Iteration**

**BACKGROUND**

<p align = 'justify'>This section is build further on arc consistency and here we replace the AC3 algorithm with a propagator iteration approach to achieve arc consistency.</p> 

In [ ]:
import itertools

**Task1**

<p align = 'justify'>Add a class that models a Boolean conjunction constraint to the code of Lab07 similar to the Boolean disjunction function implemented in there. The conjunction class should take two conjunctions of literals 𝑐1 and 𝑐2 as input and implement the constraint 𝑐1 ⇒ 𝑐2, so that satisfiability of 𝑐2 is only enforced if the conjunction 𝑐1 is also satisfiable. This can be realised by passing four lists of Boolean variables positive, negative, onlyenforceif, onlyenforceifnot to the constructor. Make sure that both implement the same interface for the CheckConstraint(x,v) method.</p>


In [ ]:
class BoolVariable():
    def __init__(self,name):
        self.name_ = name
        self.domain_ = [True,True]

    def RemoveFromDomain(self,v):
        if v:
            self.domain_[1]=False
        else:
            self.domain_[0]=False            
    
    def GetName(self):
        return self.name_
    
    def IsInList(self, list):
        for l in list:
            if self.name_ == l.name_:
                return True
        return False
    
    def GetValues(self):
        values = []
        if self.domain_[0]:
            values.append(False)
        if self.domain_[1]:
            values.append(True)
        return values

**Task 2**

<p align = 'justify'>Use the model class from Lab07 and replace the AC3 algorithm by a propagator iteration based approach to make all domains of all variables in the model arc consistent. The propagator should update the domains of all variables in each iteration with the projection:</p> 

𝐷′′(𝑥𝑖) = 𝜋{𝑥𝑖}(𝑐𝑗 ∩ 𝜋𝑋(𝑐𝑗)(𝐷′))

The result should be the same as when using the AC3 approach.

<p align = 'justify'>(Again, for the purpose of using this code in a future lab exercise, please make sure that you determine equality of variables based on their names instead of their object references, i.e. for two variables x1 and x2, do not compare them with x1==x2, but rather compare them with x1.name==x2.name)</p>

In [ ]:
class Constraint():
    def CheckConstraint(self, x,v):
        result = False            
        for t in self.combinations_:                                
            is_valid = True
            for s in self.scheme_:
                values = s.GetValues()
                if len(values)==1:
                    if not ((values[0] and s.IsInList(t)) or ((not values[0]) and (not s.IsInList(t)))):
                        is_valid = False
            if is_valid:
                if (v and (x.IsInList(t)) or ((not v) and (not x.IsInList(t)))):                                        
                    result = True
                    break
        return result        

    def Print(self):
        for t in self.combinations_:
            for x in self.scheme_:
                if (x.IsInList(t)):
                    print(x.name_, end=",")
                else:
                    print("not",x.name_, end=",")
            print()
        print()

In [ ]:
class OrConstraint(Constraint):
    def __init__(self, positive, negative):
        self.scheme_ = []
        for p in positive:
            self.scheme_.append(p)
        for n in negative:
            self.scheme_.append(n)
        self.combinations_ = self.GetAllCombinations_(positive,negative)
#        self.Print()

    def GetAllCombinations_(self,positive,negative):        
        result = []
        for length in range(len(self.scheme_)+1):
            for subset in itertools.combinations(self.scheme_,length):
                valid = False
                for x in self.scheme_:
                    if x.IsInList(subset):
                        if x.IsInList(positive):
                            valid = True
                            break
                    else: 
                        if x.IsInList(negative):
                            valid = True
                            break
                if valid:
                    result.append(subset)
        return result

In [ ]:
class AndConstraint(Constraint):
    def __init__(self, positive, negative, onlyenforceif, onlyenforceifnot):
        self.scheme_ = []
        for p in positive:
            self.scheme_.append(p)
        for n in negative:
            self.scheme_.append(n)
        for p in onlyenforceif:
            self.scheme_.append(p)
        for n in onlyenforceifnot:
            self.scheme_.append(n)
        self.combinations_ = self.GetAllCombinations_(positive,negative,onlyenforceif,onlyenforceifnot)
        # self.Print()

    def GetAllCombinations_(self,positive,negative,onlyenforceif,onlyenforceifnot):        
        result = []
        for length in range(len(self.scheme_)+1):
            for subset in itertools.combinations(self.scheme_,length):                
                enforce = True
                for x in onlyenforceif:
                    if not x.IsInList(subset):
                        enforce = False
                for x in onlyenforceifnot:
                    if x.IsInList(subset):
                        enforce = False            
                valid = True
                for x in positive:
                    if not x.IsInList(subset):
                        valid = False
                for x in negative:
                    if x.IsInList(subset):
                        valid = False
                if not enforce or valid:
                    result.append(subset)
        return result


In [ ]:
class Model():
    def __init__(self):
        self.variables_ = []
        self.constraints_ = []
        
    def AddBoolVariable(self, name):
        v = BoolVariable(name)
        self.variables_.append(v)
        return v
        
    def AddOrConstraint(self, positive, negative):
        c = OrConstraint(positive,negative)
        self.constraints_.append(c)

    def AddAndConstraint(self, positive, negative, onlyenforceif,onlyenforceifnot):
        c = AndConstraint(positive,negative,onlyenforceif,onlyenforceifnot)
        self.constraints_.append(c)

    def PrintDomains(self):
        for x in self.variables_:
            print(x.GetName(), x.GetValues())
        print()
        
    def PropagatorIteration(self):
        while True:
            change = False
            for c in self.constraints_:
                for x in c.scheme_:
                    for v in x.GetValues():
                        if not c.CheckConstraint(x,v):
                            x.RemoveFromDomain(v)
                            change = True
            if not change:
                break

In [ ]:
model = Model()

In [ ]:
x1 = model.AddBoolVariable("x1")
x2 = model.AddBoolVariable("x2")
x3 = model.AddBoolVariable("x3")
x4 = model.AddBoolVariable("x4")

In [ ]:
model.AddOrConstraint([x1,x2],[])
model.AddOrConstraint([],[x2])
model.AddOrConstraint([x3],[x1])
model.AddOrConstraint([x4],[x2])

In [ ]:
model.AddAndConstraint([x1],[x2,x4],[x3],[])

In [ ]:
model.PrintDomains()
model.PropagatorIteration()    
model.PrintDomains()

x1 [False, True]
x2 [False, True]
x3 [False, True]
x4 [False, True]

x1 [True]
x2 [False]
x3 [True]
x4 [False]

